Dec 9, 2021

Jason Cardinal Exercise Notebook Lighthouse Labs - Null Value Replacement

[https://data.compass.lighthouselabs.ca/days/w03d3/activities/459]

this notebook contains the code along notes

The next step, which is equally important, is the Null Value Replacement. We are now going to learn what to do about null values once we find them in our datasets.

## Null Value Replacement

### Why Do We Care?

Missing data in a training data set can reduce the power/fit of a model if treated as a real category, because: - Some models ignore rows with missing values effectively reducing your sample size. - It can lead to a wrong prediction or classification. - Missing values do not always mean an empty field.

Let's see a very simple example in the table below. In the left table, we have two missing values in the 'gender' column. When we check how many men and women play cricket it's gonna be 50% each.

However, when we fill in the missing values (we can easily do that based on the 'name' column) we will get the actual result: - Women – 75% play cricket - Men – 50% play cricket

https://i.imgur.com/kiqqa5w.png

Based on the table above, we can say that women are more likely to play cricket than men.

In [2]:
# import the correct packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# load the dataframe
df_train = pd.read_csv('titanic/train.csv')
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### How to Handle Missing Values

There are many different ways of dealing with missing values. It depends on the size of the data we have, the data type, or what model we want to use.

#### 1. Deleting Rows

This method commonly used to handle the null values. Here, we either delete a particular row if it has a null value for a particular feature and a particular column if it has more than 70-75% of missing values. This method is advised only when there are enough samples in the data set. One has to make sure that after we have deleted the data, there is no addition of bias. Removing the data will lead to loss of information which will not give the expected results while predicting the output.

In [6]:
df_train.dropna(inplace = True)
df_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Pros:

* Complete removal of data with missing values results in robust and highly accurate model
* Deleting a particular row or a column with no specific information is better, since it does not have a high weightage

Cons:

* Loss of information and data
* Works poorly if the percentage of missing values is high (say 30%), compared to the whole dataset

#### 2. Replacing With Mean/Median/Mode

This strategy can be applied on a feature which has numeric data like the age of a person or the ticket fare. We can calculate the mean, median or mode of the feature and replace it with the missing values. This is an approximation which can add variance to the data set. But the loss of the data can be negated by this method which yields better results compared to removal of rows and columns. Replacing with the above three approximations are a statistical approach of handling the missing values. This method is also called as leaking the data while training. Another way is to approximate it with the deviation of neighbouring values. This works better if the data is linear.

In [7]:
df_train['Age'].isnull().sum()

0

In [8]:
df_train['Age'].mean()

35.6744262295082

In [10]:
df_train['Age'].replace(np.NaN, df_train['Age'].mean()).head(15)

1     38.0
3     35.0
6     54.0
10     4.0
11    58.0
21    34.0
23    28.0
27    19.0
52    49.0
54    65.0
62    45.0
66    29.0
75    25.0
88    23.0
92    46.0
Name: Age, dtype: float64

To replace it with median and mode we can use the following to calculate the same:

In [12]:
df_train['Age'].median()

36.0

In [13]:
df_train['Age'].mode()

0    36.0
dtype: float64

Pros:

* This is a better approach when the data size is small
* It can prevent data loss which results in removal of the rows and columns

Cons:

* Imputing the approximations add variance and bias
* Works poorly compared to other multiple-imputations method

#### 3. Assigning An Unique Category

A categorical feature will have a definite number of possibilities, such as gender, for example. Since they have a definite number of classes, we can assign another class for the missing values. Here, the features Cabin and Embarked have missing values which can be replaced with a new category, say, U for ‘unknown’. This strategy will add more information into the dataset which will result in the change of variance. Since they are categorical, we need to find one hot encoding to convert it to a numeric form for the algorithm to understand it. Let us look at how it can be done in Python:

In [14]:
df_train['Cabin'].head(10)

1             C85
3            C123
6             E46
10             G6
11           C103
21            D56
23             A6
27    C23 C25 C27
52            D33
54            B30
Name: Cabin, dtype: object

In [15]:
df_train['Cabin'].fillna('U').head(10)

1             C85
3            C123
6             E46
10             G6
11           C103
21            D56
23             A6
27    C23 C25 C27
52            D33
54            B30
Name: Cabin, dtype: object

Pros:

* Less possibilities with one extra category, resulting in low variance after one hot encoding — since it is categorical
* Negates the loss of data by adding an unique category

Cons:

* Adds less variance
* Adds another feature to the model while encoding, which may result in poor performance

#### 4. Predicting The Missing Values

Using the features which do not have missing values, we can predict the nulls with the help of a machine learning algorithm. This method may result in better accuracy, unless a missing value is expected to have a very high variance. We will be using linear regression to replace the nulls in the feature ‘age’, using other available features. One can experiment with different algorithms and check which gives the best accuracy instead of sticking to a single algorithm.

In [17]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

data_with_null = df_train[['PassengerId', 'Pclass', 'Survived', 'SibSp', 'Parch', 'Fare', 'Age']].dropna()

date_without_null = data_with_null.dropna()

# All features except AGE

train_data_x = date_without_null.iloc[:,:6]

# Only Age

train_data_y = date_without_null.iloc[:,6]

# Training with the available data

linreg.fit(train_data_x, train_data_y)

# Predict for the whole dataset and replace only the missing values later.

test_data = data_with_null.iloc[:,:6]
age_predicted['Age'] = pd.DataFrame(linreg.predict(test_data))

# Lets replace only the missing values

data_with_null.Age.fillna(age_predicted.Age, inplace= True)

NameError: name 'age_predicted' is not defined

Pros:

* Imputing the missing variable is an improvement as long as the bias from the same is smaller than the omitted variable bias
* Yields unbiased estimates of the model parameters
*
Cons:

* Bias also arises when an incomplete conditioning set is used for a categorical variable
* Considered only as a proxy for the true values

#### 5. Using Algorithms Which Support Missing Values

KNN is a machine learning algorithm which works on the principle of distance measure. This algorithm can be used when there are nulls present in the dataset. While the algorithm is applied, KNN considers the missing values by taking the majority of the K nearest values. In this particular dataset, taking into account the person’s age, sex, class etc, we will assume that people having same data for the above mentioned features will have the same kind of fare.

Unfortunately, the SciKit Learn library for the K – Nearest Neighbour algorithm in Python does not support the presence of the missing values.

Another algorithm which can be used here is RandomForest. This model produces a robust result because it works well on non-linear and the categorical data. It adapts to the data structure taking into consideration of the high variance or the bias, producing better results on large datasets.

Pros:

* Does not require creation of a predictive model for each attribute with missing data in the dataset
* Correlation of the data is neglected

Cons:

* Is a very time consuming process and it can be critical in data mining where large databases are being extracted
* Choice of distance functions can be Euclidean, Manhattan etc. which is do not yield a robust result

#### Conclusion

Every dataset we come across will almost have some missing values which need to be dealt with. But handling them in an intelligent way and giving rise to robust models is a challenging task. We have gone through a number of ways in which nulls can be replaced. It is not necessary to handle a particular dataset in one single manner. One can use various methods on different features depending on how and what the data is about. Having a small domain knowledge about the data is important, which can give you an insight about how to approach the problem.


Let's go back to our House Price Prediction Project. In this part, we are going to handle missing variables in the dataset.

### Prediction of House Prices: Null Value Replacement

We can continue in the same notebook as in the activityOutlier Detection.

As a first step, we will check whether we have some columns with missing values.

In [18]:
# missing data
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
Embarked,0,0.0
Cabin,0,0.0
Fare,0,0.0
Ticket,0,0.0
Parch,0,0.0
SibSp,0,0.0
Age,0,0.0
Sex,0,0.0
Name,0,0.0
Pclass,0,0.0


There are a couple of columns, where most of the values are missing. We can get rid of these since they don't have any value for us.

In [20]:
# drop 5 columns with the biggest ratio of null values
to_drop = missing_data.head(5).index.tolist()
df_train.drop(to_drop, axis=1, inplace=True)

KeyError: "['Embarked' 'Cabin' 'Fare' 'Ticket' 'Parch'] not found in axis"

We don't want to remove more than 5 columns because the rest don't have a lot of missing values.


In [21]:
# extract the names of columns with missing values
cols_with_missing = missing_data[missing_data.Percent > 0].index.tolist()
# remove column names that are already removed from dataset
missing_cols = list(set(cols_with_missing) - set(to_drop))
# check the datatype
df_train.dtypes[missing_cols]

Series([], dtype: object)

We can see that we have a few numeric variables and some with text inside. First, we will take a look at the numeric ones.

### Numeric Variables

We will create a separate column to keep the information whether the value was missing. This way, we will have this information available in this column after we replace the value in the original one.

In [22]:
num_cols_with_missing = df_train.dtypes[missing_cols][df_train.dtypes[missing_cols] == 'float'].index.tolist()

# create new variable with the information that it was missing

for cl in num_cols_with_missing:
    df_train[cl + "_missing_ind"] = 0
    df_train.loc[df_train[cl].isnull(), cl + "_missing_ind"] = 1

Now we can work on the Null value replacement.

In [ ]:
df_train["LotFrontage"] = df_train["LotFrontage"].fillna(df_train["LotFrontage"].mean())
df_train["GarageYrBlt"] = df_train["GarageYrBlt"].fillna(df_train["GarageYrBlt"].min())
df_train["MasVnrArea"] = df_train["MasVnrArea"].fillna(0)

* LotFrontage – We replace the missing values with the mean.
* GarageYrBlt – If the house has a garage and the year is missing, we assume it's the minimum value available.
* MasVnrArea – If the veneer area is missing, we assume it's 0.

### Object (String) Variables

Find the variables with the dtype == "object" and having at least one missing value.

In [23]:
# 2. for categorical variables:
df_train.dtypes[missing_cols][df_train.dtypes[missing_cols] == 'object']
cat_cols_with_missing = df_train.dtypes[missing_cols][df_train.dtypes[missing_cols] == 'object'].index.tolist()

With the help of the data documentation we have, we can figure out that the missing values in Garage and Basement variables mean no garage and no basement respectively. Therefore, we will replace the missing values with "None".

In [ ]:
# GarageFinish : data description says NA means "no garage"
df_train["GarageFinish"] = df_train["GarageFinish"].fillna("None")
# GarageCond : data description says NA means "no garage"
df_train["GarageCond"] = df_train["GarageCond"].fillna("None")
# GarageQual : data description says NA means "no garage"
df_train["GarageQual"] = df_train["GarageQual"].fillna("None")
# GarageType : data description says NA means "no garage"
df_train["GarageType"] = df_train["GarageType"].fillna("None")

# BsmtExposure : data description says NA means "no basement"
df_train["BsmtExposure"] = df_train["BsmtExposure"].fillna("None")
# BsmtFinType2 : data description says NA means "no basement"
df_train["BsmtFinType2"] = df_train["BsmtFinType2"].fillna("None")
# BsmtFinType1 : data description says NA means "no basement"
df_train["BsmtFinType1"] = df_train["BsmtFinType1"].fillna("None")
# BsmtCond : data description says NA means "no basement"
df_train["BsmtCond"] = df_train["BsmtCond"].fillna("None")
# BsmtQual : data description says NA means "no basement"
df_train["BsmtQual"] = df_train["BsmtQual"].fillna("None")

The information about Electrical and MasVnrType is missing in the documentation. Since we are dealing with categorical variables, we will create a new category for a missing value.

In [ ]:
df_train["Electrical"] = df_train["Electrical"].fillna("Empty")
df_train["MasVnrType"] = df_train["MasVnrType"].fillna("Empty")

Finally, we can run the missing values check once more to see if we have filled in all the data.